In [1]:
import os 
import re
import tweepy as tw
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
consumer_key = 'your_consumer_key'
consumer_secret = 'your_consumer_secret'
bearer_token = 'your_bearer_token'
access_token = 'your_access_token'
access_token_secret = 'your_access_token_secret'
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
analyzer = SentimentIntensityAnalyzer()
def text_sentiment(text):
    vs = analyzer.polarity_scores(text)
    if vs.get("compound") >= 0.05:
        return 1
    elif vs.get("compound") <= -0.05:
        return -1
    else:
        return 0

# Existing players in GT roster with twitter account

In [4]:
ingt = pd.read_csv("rivals.csv")
ingt.head()
ihavetw = pd.read_csv("existing_gt_players.csv")
ihavetw.head()

,name,twitter
0,djimon brooks,https://twitter.com/djimoskie_69
1,juanyeh thomas,https://twitter.com/STG_Yeh1
2,tariq carpenter,https://twitter.com/STG_Riq2
3,tre swilling,https://twitter.com/SwillingTre
4,tobias oliver,https://twitter.com/tobiasoliver_8


In [5]:
ingttw = ingt.dropna(axis = 0)
ingttw["name"] = ingttw["NAME "].apply(lambda x: str.lower(x))
withtw = pd.merge(ingttw, ihavetw, on = 'name', how = 'left')


<ipython-input-5-03e5cb20c82a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingttw["name"] = ingttw["NAME "].apply(lambda x: str.lower(x))


In [ ]:
withtw.to_csv("twitter.csv", index = False)

In [6]:
ihavetw = pd.read_csv("twitter.csv")

In [7]:
ihavetw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  41 non-null     int64  
 1   NAME        41 non-null     object 
 2   POS         41 non-null     object 
 3   LOCATION    41 non-null     object 
 4   HT          41 non-null     object 
 5   WT          41 non-null     int64  
 6   STARS       41 non-null     int64  
 7   RATING      41 non-null     float64
 8   COMMITTED   41 non-null     object 
 9   account     41 non-null     int64  
 10  name        41 non-null     object 
 11  twitter     41 non-null     object 
dtypes: float64(1), int64(4), object(7)
memory usage: 4.0+ KB


In [8]:
for url in ihavetw["twitter"].values:
    ihavetw.loc[ihavetw["twitter"] == url, "twitter"] = re.sub("https://twitter.com/","", url)
print(ihavetw.head())

   Unnamed: 0             NAME   POS           LOCATION     HT   WT  STARS  \
0           0  Khatavian Franks  LB       Fairburn, GA   6'3"   203      3   
1           1      Jeffrey Sims  QB   Jacksonville, FL   6'3"   195      4   
2           2        Jared Ivey  DE        Suwanee, GA   6'6"   215      3   
3           3     Paula Vaipulu  OL    Channelview, TX   6'4"   303      3   
4           4       Ryan Spiers  OL         Biloxi, MS   6'4"   275      2   

   RATING   COMMITTED  account              name         twitter  
0     5.6  2019-12-18        1  khatavian franks     khatavian_4  
1     5.8  2019-12-18        1      jeffrey sims     JeffSims_10  
2     5.7  2019-12-11        1        jared ivey  jared_ivey1595  
3     5.5  2019-10-31        1     paula vaipulu          PTV_70  
4     5.4  2019-09-08        1       ryan spiers    ryanspiers71  


In [9]:
ihavetw["state"] = ihavetw["LOCATION"].str.split(',', expand = True)[1].str.strip()
ihavetw["POS"] = ihavetw["POS"].str.strip()
ihavetw["state"] = ihavetw["state"].apply(lambda x: "INT" if str(x) == 'null' else x)
ihavetw["COMMITTED"] = pd.to_datetime(ihavetw["COMMITTED"])

In [10]:
heights = ihavetw["HT"].str.split(r'(\'|\")', expand = True)
ihavetw["height_in"] = heights[0].astype('float') * 12 + heights[2].astype('float')

In [11]:
userid = ihavetw["twitter"].values
ihavetw[["sentiment", "neu", "pos","neg"]] = 0

In [12]:
try:
    api.get_user('__TKC11')
except Exception as error:
    print(error)

[{'code': 63, 'message': 'User has been suspended.'}]


In [13]:
count = 25
for name in userid:
    totsent, neutral, posi, nega = [], [] , [] ,[]
#     print(name)
    try:
        if not api.get_user(name).protected:
            datemax = ihavetw.loc[ihavetw["twitter"] == name, "COMMITTED"]
            tweets = tw.Cursor(api.user_timeline, id = name, max = datemax, count = count)
            for a in tweets.items(count):
                txt = a.text
                totsent.append(text_sentiment(txt))
                feel = analyzer.polarity_scores(txt)
                neutral.append(feel.get('neu'))
                posi.append(feel.get('pos'))
                nega.append(feel.get('neg'))
            ihavetw.loc[ihavetw["twitter"] == name, "sentiment"] = np.mean(totsent)
            ihavetw.loc[ihavetw["twitter"] == name, "neu"] = np.mean(neutral)
            ihavetw.loc[ihavetw["twitter"] == name, "pos"] = np.mean(posi)
            ihavetw.loc[ihavetw["twitter"] == name, "neg"] = np.mean(nega)
    except Exception as error:
        print(error, name)
        ihavetw.loc[ihavetw["twitter"] == name, "sentiment"] = -1
        ihavetw.loc[ihavetw["twitter"] == name, "neu"] = 0
        ihavetw.loc[ihavetw["twitter"] == name, "pos"] = 0
        ihavetw.loc[ihavetw["twitter"] == name, "neg"] = -1

[{'code': 63, 'message': 'User has been suspended.'}] __TKC11


In [14]:
ihavetw.drop(columns = ['Unnamed: 0', 'account', 'name'], inplace = True)

In [15]:
ihavetw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   NAME       41 non-null     object        
 1   POS        41 non-null     object        
 2   LOCATION   41 non-null     object        
 3   HT         41 non-null     object        
 4   WT         41 non-null     int64         
 5   STARS      41 non-null     int64         
 6   RATING     41 non-null     float64       
 7   COMMITTED  41 non-null     datetime64[ns]
 8   twitter    41 non-null     object        
 9   state      41 non-null     object        
 10  height_in  41 non-null     float64       
 11  sentiment  41 non-null     float64       
 12  neu        41 non-null     float64       
 13  pos        41 non-null     float64       
 14  neg        41 non-null     float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(6)
memory usage: 4.9+ KB


In [17]:
offense = ['WR', 'RB', 'QB', 'TE', 'LT', 'LG', 'C', 'RG', 'RT', 'OL', 'ATH']
defense = ['CB', 'FS','DE','DT','LB', 'SS', 'DB']
ihavetw["part"] = ihavetw["POS"].apply(lambda x: 'defense' if x in defense else 'offense')

In [18]:
ihavetw.head(10)

,NAME,POS,LOCATION,HT,WT,STARS,RATING,COMMITTED,twitter,state,height_in,sentiment,neu,pos,neg,part
0,Khatavian Franks,LB,"Fairburn, GA","6'3""",203,3,5.6,2019-12-18,khatavian_4,GA,75.0,0.16,0.85028,0.09476,0.05496,defense
1,Jeffrey Sims,QB,"Jacksonville, FL","6'3""",195,4,5.8,2019-12-18,JeffSims_10,FL,75.0,0.36,0.82228,0.12556,0.05220,offense
2,Jared Ivey,DE,"Suwanee, GA","6'6""",215,3,5.7,2019-12-11,jared_ivey1595,GA,78.0,0.32,0.88184,0.11260,0.00560,defense
3,Paula Vaipulu,OL,"Channelview, TX","6'4""",303,3,5.5,2019-10-31,PTV_70,TX,76.0,0.28,0.82132,0.12464,0.05408,offense
4,Ryan Spiers,OL,"Biloxi, MS","6'4""",275,2,5.4,2019-09-08,ryanspiers71,MS,76.0,0.44,0.81844,0.16596,0.01560,offense
5,Avery Boyd,WR,"Tallahassee, FL","6'2""",198,3,5.6,2019-08-24,ABoyd723,FL,74.0,0.20,0.88648,0.08988,0.02368,offense
6,Wing Green,OL,"Leesburg, GA","6'7""",270,3,5.5,2019-06-25,WingGreenJr,GA,79.0,0.36,0.83560,0.13124,0.03316,offense
7,Khaya Wright,DE,"Miami, FL","6'3""",200,3,5.6,2019-06-25,Khayawright18,FL,75.0,0.00,1.00000,0.00000,0.00000,defense
8,Kyle Kennard,DE,"Atlanta, GA","6'6""",215,4,5.6,2019-06-12,KyleKennard1,GA,78.0,0.36,0.71896,0.17136,0.10964,defense
9,Jahmyr Gibbs,RB,"Dalton, GA","5'11""",194,3,5.9,2019-05-25,Jahmyr_Gibbs1,GA,71.0,0.56,0.82112,0.16268,0.01624,offense


In [20]:
ihavetw.to_csv("current+twit.csv", index = False)

In [21]:
defensename = ihavetw[ihavetw["part"] == 'defense']["NAME "].str.split(' ', expand = True)
offensename = ihavetw[ihavetw["part"] == 'offense']["NAME "].str.split(' ', expand = True)

In [22]:
offensename

,0,1
1,Jeffrey,Sims
3,Paula,Vaipulu
4,Ryan,Spiers
5,Avery,Boyd
6,Wing,Green
9,Jahmyr,Gibbs
11,Cade,Kootsouradis
14,Ryan,King
15,Billy,Ward
18,Jamious,Griffin


In [23]:
import glob as glob

In [24]:
dfh = glob.glob("Players_by_cat\players_defense*.csv")
dffile = []
for i in dfh:
    data = pd.read_csv(i)
    dffile.append(data)

In [25]:
ofh = glob.glob("Players_by_cat\players_offense*.csv")
offile = []
for i in ofh:
    data = pd.read_csv(i)
    offile.append(data)

In [26]:
offensetotal = pd.concat(offile)
oflastname = offensetotal["name"].str.split(' ', expand = True)[1]
offirstname = offensetotal["name"].str.split(' ', expand = True)[0]
[x for x in offensename[1] if x in oflastname.values]
ofsortlast = [offensetotal[oflastname.values == x] for x in offensename[1] if x in oflastname.values]
ofsortfirst = [offensetotal[offirstname.values == x] for x in offensename[0] if x in offirstname.values]  

In [27]:
defensetotal = pd.concat(dffile)
dflastname = defensetotal["name"].str.split(' ', expand = True)[1]
dffirstname = defensetotal["name"].str.split(' ', expand = True)[0]

dfsortlast = [defensetotal[dflastname.values == x] for x in defensename[1] if x in dflastname.values]
dfsortfirst = [defensetotal[dffirstname.values == x] for x in defensename[0] if x in dffirstname.values]  

In [28]:
ofsortlast = pd.concat(ofsortlast)
ofsortfirst = pd.concat(ofsortfirst)
dfsortlast = pd.concat(dfsortlast)
dfsortfirst = pd.concat(dfsortfirst)

In [29]:
ofsortlast.to_csv("offensefirst.csv", index = False)
ofsortfirst.to_csv("offenselast.csv", index = False)

In [30]:
dfsortlast.to_csv("defensefirst.csv", index = False)
dfsortfirst.to_csv("defenselast.csv", index = False)